<a href="https://colab.research.google.com/github/ilucas75/BTflow/blob/master/RD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#install
!pip install pdfminer.six
!pip install spacy

In [0]:
!python -m spacy download fr_core_news_sm
!python -m spacy download fr_core_news_md

In [0]:
from google.colab import files

uploaded = files.upload()


Saving 6900C022_220C0096.pdf to 6900C022_220C0096.pdf


In [0]:
ls

1360C022_220C0631.pdf  sample_data/


In [0]:
import io
 
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [0]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
 
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
 
        text = fake_file_handle.getvalue()
 
    # close open handles
    converter.close()
    fake_file_handle.close()
 
    if text:
        return text

In [0]:
for f in uploaded.keys():
  pdftotext=extract_text_from_pdf(f)

In [0]:
print(pdftotext)

 220C0631-FR0013181864-FS0171              220C0631 FR0013181864-FS0171 17 février 2020   Déclaration de franchissement de seuils (article L. 233-7 du code de commerce)    CGG  (Euronext Paris)   Par courrier reçu le 17 février 2020, la société Morgan Stanley & Co International plc1 (25 Cabot Square, Canary Wharf, Londres E14 4QA, Royaume Uni), a déclaré avoir franchi individuellement en hausse, le 11 février 2020, les seuils de 5% du capital et des droits de vote de la société CGG et détenir individuellement 37 182 936 actions CGG représentant autant de droits de vote, soit 5,24% du capital et des droits de vote de cette société2.  Ce franchissement de seuils résulte d’une acquisition d’actions CGG hors marché.  À cette occasion, la société Morgan Stanley Corp. (c/o The Corporation Trust Company (DE), Corporation Trust Center, 1209 Orange Street, Wilmington, Delaware DE 19801, Etats-Unis) a précisé n’avoir franchi aucun seuil et détenir, indirectement, 37 193 524 actions CGG représent

In [0]:
!pip install tabula-py

     |████████████████████████████████| 10.4MB 4.1MB/s 


In [0]:
from tabula import read_pdf

df = read_pdf("6900C022_220C0096.pdf")

'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: Mar 22, 2020 3:38:19 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F1 are not implemented in PDFBox and will be ignored
Mar 22, 2020 3:38:20 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F2 are not implemented in PDFBox and will be ignored
Mar 22, 2020 3:38:20 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F3 are not implemented in PDFBox and will be ignored



In [0]:
for d in df:
    print(d)

Empty DataFrame
Columns: [Unnamed: 0]
Index: []
Empty DataFrame
Columns: [Unnamed: 0, Unnamed: 1, Unnamed: 2, Unnamed: 3]
Index: []
Empty DataFrame
Columns: [Unnamed: 0]
Index: []
Empty DataFrame
Columns: [Unnamed: 0, Unnamed: 1, Unnamed: 2]
Index: []
Empty DataFrame
(Euronext)]
Index: []
            Unnamed: 0  Unnamed: 1  ... Droits de vote % droits de vote
0        Olivier Ginon         NaN  ...         10 290            0,02%
1         Le Grand Rey         NaN  ...        151 003            0,34%
2         Ferme d'Anna         NaN  ...         60 394            0,14%
3  Total Olivier Ginon         NaN  ...        221 687            0,50%
4         Olivier Roux         NaN  ...          8 400            0,02%
5             Polygone         NaN  ...     27 994 373           63,36%
6     Total fondateurs         NaN  ...     28 224 460           63,89%
7               Sofina         NaN  ...      6 740 739           15,26%
8        Total concert         NaN  ...     34 965 199        

In [0]:
import spacy
import fr_core_news_sm
import fr_core_news_md

#nlp = fr_core_news_sm.load()
nlp = fr_core_news_md.load()

doc = nlp(pdftotext)
for sent in doc.sents:
    print(sent)



 220C0631-FR0013181864-FS0171              220C0631 FR0013181864-FS0171 17 février 2020   
Déclaration de franchissement de seuils (article L. 233
-7 du code de commerce)    CGG  (Euronext Paris)   Par courrier reçu le 17 février 2020, la société Morgan Stanley & Co International plc1 (25 Cabot Square, Canary Wharf, Londres E14 4QA, Royaume Uni), a déclaré avoir franchi individuellement en hausse, le 11 février 2020, les seuils de 5% du capital et des droits de vote de la société CGG et détenir individuellement 37 182 936 actions CGG représentant autant de droits de vote, soit 5,24% du capital et des droits de vote de cette société2.  
Ce franchissement de seuils résulte d’une acquisition d’actions CGG hors marché.  
À cette occasion, la société Morgan Stanley Corp.
(c/o The Corporation Trust Company (DE), Corporation Trust Center, 1209 Orange Street, Wilmington, Delaware DE 19801, Etats
-Unis) a précisé n’avoir franchi aucun seuil et détenir, indirectement, 37 193 524 actions CGG repr

In [0]:
for ent in doc.ents:
    print(ent.text, ent.label_)


In [0]:
from spacy import displacy

In [0]:
displacy.render(doc, style='ent', jupyter=True)

In [0]:
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.label_, chunk.root.text)
 

220C0170-FR0012407096-OP028-AS12-RO02            220C0170 FR0012407096-OP028-AS12-RO02 14 janvier 2020    NP 220C0170-FR0012407096-OP028-AS12-RO02
- Mise en œuvre du retrait obligatoire visant les actions de la société.   NP Mise
- Maintien de la suspension de la cotation des actions de la société.     NP Maintien
Euronext Paris NP Euronext
Le 13 janvier 2020 NP janvier
l’Autorité des marchés financiers NP Autorité
d’achat simplifiée NP achat
les actions ITS GROUP NP actions
la société ITS Participations NP société
actions ITS GROUP NP actions
de droits de vote NP droits
des droits de vote de cette société2 NP droits
cf NP cf
D&I 220C0149 en date du 13 janvier 2020).   NP D&I
Les actions ITS GROUP non présentées à l’offre par les actionnaires minoritaires NP actions
de 366 396 actions NP actions
483 droits de vote, NP droits
soit 4,65% du capital et 4,92% des droits de vote de cette société2 NP %
2- Par courrier NP 2-
de la société NP société
ITS Participations NP Participations
l’Auto